# ACS2 in Multiplexer

In [1]:
%matplotlib inline

# General
from __future__ import unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Logger
import logging
logging.basicConfig(level=logging.WARN)

# ALCS + Maze environment
import sys
sys.path.append('/Users/khozzy/Projects/pyalcs')
sys.path.append("/Users/khozzy/Projects/openai-maze-envs")

# Enable automatic module reload
%load_ext autoreload
%autoreload 2

# Load PyALCS module
from alcs import ACS2, ACS2Configuration

# Load environments
import gym
import gym_multiplexer

## Multiplexer

In [2]:
mp = gym.make('boolean-multiplexer-6bit-v0')
situation = mp.reset()
mp.render()

'1101010'

In [3]:
state, reward, done, _ = mp.step(0)

print("New state: {}, reward: {}, is done: {}".format(state, reward, done))

New state: 1101010, reward: 0, is done: True


In [4]:
mp.render()

'1101010'

### Evaluation
Calculate knowledge - check if any of the reliable classifiers gives correct answer.

In [5]:
from gym_multiplexer.utils import get_correct_answer

def calculate_knowledge(env, population, ctrl_bits):
    p1 = env.render()  # state after executing action
    p0 = p1[:-1] + '0'  # initial state
    correct_answer = get_correct_answer(p0, ctrl_bits)  # true action
    
    reliable_classifiers = [c for c in population if c.is_reliable()]
    
    return any([1 for cl in reliable_classifiers if cl.predicts_successfully(p0, correct_answer, p1)])

## Go agent, go...

In [6]:
cfg = ACS2Configuration(mp.env.observation_space.n, 2, do_ga=True)
agent = ACS2(cfg)

population, metrics = agent.explore(mp, 10000)

In [7]:
print("Population size: {}".format(metrics[-1]['agent']['population']))
print("Reliable size: {}".format(metrics[-1]['agent']['reliable']))
print("Knowledge: {:.2f}%".format(calculate_knowledge(mp, population, 2)))
print(metrics[-1])

Population size: 117
Reliable size: 39
Knowledge: 1.00%
{'agent': {'population': 117, 'numerosity': 338, 'reliable': 39, 'fitness': 475.4286728999321, 'trial': 9999, 'steps': 1, 'total_steps': 10000}, 'environment': None}


In [10]:
p1 = mp.render()
p0 = p1[:-1] + '0'
correct_answer = get_correct_answer(p0, 2)

reliable_classifiers = [c for c in population if c.is_reliable()]

for cl in reliable_classifiers:
    print("{}, q: {:.2f}, exp: {:.2f}".format(cl, cl.q, cl.exp))

0101100-1-######1 @ 0x10673d668, q: 1.00, exp: 138.00
000100#-1-####### @ 0x1067b72e8, q: 0.98, exp: 57.00
001001#-0-####### @ 0x1067b7550, q: 0.98, exp: 56.00
000111#-1-####### @ 0x1067b7cf8, q: 0.98, exp: 56.00
001000#-0-####### @ 0x1067b7dd8, q: 0.98, exp: 56.00
10##0##-1-####### @ 0x10322add8, q: 1.00, exp: 353.00
010110#-0-####### @ 0x10213dd30, q: 0.96, exp: 44.00
111001#-0-####### @ 0x106827908, q: 0.97, exp: 53.00
11###1#-0-####### @ 0x1068278d0, q: 1.00, exp: 378.00
01#1###-1-######1 @ 0x106827be0, q: 1.00, exp: 872.00
1010010-0-######1 @ 0x106827e10, q: 1.00, exp: 131.00
10#1010-0-######1 @ 0x1068a21d0, q: 1.00, exp: 194.00
10##1##-0-####### @ 0x1068a2860, q: 1.00, exp: 360.00
001####-0-####### @ 0x1068a2be0, q: 1.00, exp: 377.00
010100#-0-####### @ 0x1068a2cc0, q: 0.93, exp: 37.00
01#1###-0-####### @ 0x106920320, q: 1.00, exp: 343.00
100#10#-1-######1 @ 0x1069206a0, q: 1.00, exp: 209.00
10##0##-0-######1 @ 0x106920ac8, q: 1.00, exp: 858.00
000####-0-######1 @ 0x10696a400, q:

In [9]:
print("p0: {}".format(p0))
print("p1: {}".format(p1))
print("correct: {}".format(correct_answer))

p0: 0000110
p1: 0000111
correct: 0


In [10]:
nr_correct = sum([1 for cl in reliable_classifiers if cl.predicts_successfully(p0, correct_answer, p1) and c.condition.does_match(p0)])
nr_correct

NameError: name 'c' is not defined